In [2]:
import pandas as pd

master_url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
files = pd.read_csv(master_url, sep=" ", header=None)

files = files[[2]]
files.columns = ["url"]
files.head()


/tmp/ipython-input-2266250146.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  files = pd.read_csv(master_url, sep=" ", header=None)


,url
0,http://data.gdeltproject.org/gdeltv2/201502182...
1,http://data.gdeltproject.org/gdeltv2/201502182...
2,http://data.gdeltproject.org/gdeltv2/201502182...
3,http://data.gdeltproject.org/gdeltv2/201502182...
4,http://data.gdeltproject.org/gdeltv2/201502182...


In [3]:
day = "20230309"

gkg_files = files[
    files["url"].str.contains(day) &
    files["url"].str.contains("gkg.csv.zip")
]

gkg_files.head()


,url
830462,http://data.gdeltproject.org/gdeltv2/202303090...
830465,http://data.gdeltproject.org/gdeltv2/202303090...
830468,http://data.gdeltproject.org/gdeltv2/202303090...
830471,http://data.gdeltproject.org/gdeltv2/202303090...
830474,http://data.gdeltproject.org/gdeltv2/202303090...


In [4]:
import requests, zipfile, io

def count_mentions(url, keyword):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    fname = z.namelist()[0]
    df = pd.read_csv(z.open(fname), sep="\t", header=None, encoding='latin-1')
    return df[14].str.contains(keyword, case=False, na=False).sum()

In [5]:
url = gkg_files.iloc[0]["url"]  # prendiamo il primo file valido

count = count_mentions(url, "Silicon Valley Bank")

print("SVB mentions:", count)


SVB mentions: 0


In [6]:
from datetime import datetime, timedelta

results = []

start = datetime(2023, 3, 6)
end = datetime(2023, 3, 15)

current = start
while current <= end:
    day = current.strftime("%Y%m%d")

    daily_files = files[
        files["url"].str.contains(day) &
        files["url"].str.contains("gkg.csv.zip")
    ]

    if len(daily_files) > 0:
        url = daily_files.iloc[0]["url"]
        count = count_mentions(url, "Silicon Valley Bank")
        results.append({"day": day, "gdelt_volume": count})

    current += timedelta(days=1)

df_gdelt_svb = pd.DataFrame(results)
df_gdelt_svb["day"] = pd.to_datetime(df_gdelt_svb["day"])
df_gdelt_svb


,day,gdelt_volume
0,2023-03-06,0
1,2023-03-07,0
2,2023-03-08,0
3,2023-03-09,0
4,2023-03-10,1
5,2023-03-11,40
6,2023-03-12,20
7,2023-03-13,47
8,2023-03-14,55
9,2023-03-15,46


In [7]:
df_gdelt_svb.to_csv("gdelt_svb_volume_free.csv", index=False)


In [8]:
#Netflix
from datetime import datetime, timedelta

start = datetime(2022, 4, 19)
end = datetime(2022, 5, 3)

dates = []
current = start
while current <= end:
    dates.append(current.strftime("%Y%m%d"))
    current += timedelta(days=1)

dates


['20220419',
 '20220420',
 '20220421',
 '20220422',
 '20220423',
 '20220424',
 '20220425',
 '20220426',
 '20220427',
 '20220428',
 '20220429',
 '20220430',
 '20220501',
 '20220502',
 '20220503']

In [ ]:
results = []

for d in dates:
    daily_files = files[
        files["url"].str.contains(d) &
        files["url"].str.contains("gkg.csv.zip")
    ]

    if len(daily_files) > 0:
        url = daily_files.iloc[0]["url"]
        count = count_mentions(url, "Netflix")
        results.append({"day": d, "gdelt_volume": count})
    else:
        results.append({"day": d, "gdelt_volume": 0})

In [9]:
df_gdelt_netflix = pd.DataFrame(results)
df_gdelt_netflix["day"] = pd.to_datetime(df_gdelt_netflix["day"])
df_gdelt_netflix

,day,gdelt_volume
0,2023-03-06,0
1,2023-03-07,0
2,2023-03-08,0
3,2023-03-09,0
4,2023-03-10,1
5,2023-03-11,40
6,2023-03-12,20
7,2023-03-13,47
8,2023-03-14,55
9,2023-03-15,46


In [10]:
df_gdelt_netflix.to_csv("gdelt_netflix_volume_free.csv", index=False)
print("Salvato: gdelt_netflix_volume_free.csv")


Salvato: gdelt_netflix_volume_free.csv


In [11]:
import pandas as pd

# Carica i due dataset GDELT
df_svb = pd.read_csv("/content/gdelt_svb_volume_free.csv")
df_netflix = pd.read_csv("/content/gdelt_netflix_volume_free.csv")

In [12]:
def normalize_to_percentage(df, value_col):
    max_value = df[value_col].max()
    df["gdelt_index_pct"] = (df[value_col] / max_value) * 100
    return df


In [13]:
df_gdelt_svb = normalize_to_percentage(df_gdelt_svb, "gdelt_volume")
df_gdelt_svb

,day,gdelt_volume,gdelt_index_pct
0,2023-03-06,0,0.000000
1,2023-03-07,0,0.000000
2,2023-03-08,0,0.000000
3,2023-03-09,0,0.000000
4,2023-03-10,1,1.818182
5,2023-03-11,40,72.727273
6,2023-03-12,20,36.363636
7,2023-03-13,47,85.454545
8,2023-03-14,55,100.000000
9,2023-03-15,46,83.636364


In [14]:
df_netflix = normalize_to_percentage(df_netflix, "gdelt_volume")
df_netflix


,day,gdelt_volume,gdelt_index_pct
0,2023-03-06,0,0.000000
1,2023-03-07,0,0.000000
2,2023-03-08,0,0.000000
3,2023-03-09,0,0.000000
4,2023-03-10,1,1.818182
5,2023-03-11,40,72.727273
6,2023-03-12,20,36.363636
7,2023-03-13,47,85.454545
8,2023-03-14,55,100.000000
9,2023-03-15,46,83.636364


In [16]:
df_gdelt_svb.to_csv("gdelt_svb_index_pct.csv", index=False)
df_netflix.to_csv("gdelt_netflix_index_pct.csv", index=False)